In [1]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ['TRANSFORMERS_CACHE'] = '/data/Models/'

In [2]:
from datasets import load_dataset

ds = load_dataset("Malikeh1375/medical-question-answering-datasets", "all-processed")

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', '__index_level_0__'],
        num_rows: 246678
    })
})

In [18]:
ds['train']['input'][0]

'Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ?'

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [5]:
tokenizer.special_tokens_map

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}

In [6]:
instruction = ds['train']['instruction'][:1000]
user = ds['train']['input'][:1000]
assistant = ds['train']['output'][:1000]

In [7]:
data = []
for i in range((1000)):
    temp_data = []
    
    #temp_data.append({"role": "system", "content":instruction[i]})
    temp_data.append({"role": "user", "content":user[i]})
    temp_data.append({"role": "assistant", "content":assistant[i]})
    data.append(temp_data)

In [42]:
data[0]

[{'role': 'system',
  'content': "If you are a doctor, please answer the medical questions based on the patient's description."},
 {'role': 'user',
  'content': 'Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ?'},
 {'role': 'assistant',
  'content': "hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but le

In [27]:
type(data[0])

dict

In [29]:
messages = [ {
  'role':'user',
  'content': 'Hey, are you free? I need someone to talk to.',
},
        {
  'role':'assistant',
  'content': "Of course, I'm here to help. Feel free to share what's on your mind and I'll listen and try to provide a supportive and understanding response. What's been going on?",
  
        },
        {
  'role':'user',
  'content': "I've been feeling really down lately. I'm struggling with my mental health and it's been hard to cope. I feel overwhelmed and alone.",
        }   ]

In [45]:
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [38]:
tokenizer_chat = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-8B")

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
outputs = tokenizer.apply_chat_template(data[0], tokenize="False")

In [51]:
mistral_tokenizer.chat_template

"{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [54]:
tokenizer.decode(outputs, skip_special_tokens=True)

"[INST] Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ? [/INST] hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but left untreated he slowly modifies the heart. according to heart rhythm, the normal rate is between 50-100 beat for minute. when it exceeds 100 we talk about sinus tachycardia. this might h

In [55]:
data[0]

[{'role': 'user',
  'content': 'Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ?'},
 {'role': 'assistant',
  'content': "hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but left untreated he slowly modifies the heart. according to heart rhythm, the normal rate is between 50-100 beat for minute. when it e

In [8]:
import pandas as pd
columns = ['content']
df = pd.DataFrame(columns=columns)
for i in range(len(data)):
    df.append(data[i])

/tmp/ipykernel_519078/3762798196.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(data[i])
/tmp/ipykernel_519078/3762798196.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(data[i])
/tmp/ipykernel_519078/3762798196.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(data[i])
/tmp/ipykernel_519078/3762798196.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(data[i])
/tmp/ipykernel_519078/3762798196.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(data[i])
/tmp/ipykernel_519078/3762798196.py

In [9]:
df['content'] = data

In [10]:
df

,content
0,"[{'role': 'user', 'content': 'Hey Just wonderi..."
1,"[{'role': 'user', 'content': 'I had a brain an..."
2,"[{'role': 'user', 'content': 'When to seek urg..."
3,"[{'role': 'user', 'content': 'Hi My problem is..."
4,"[{'role': 'user', 'content': 'Im having horrib..."
...,...
995,"[{'role': 'user', 'content': 'hi,am 31yrs,i we..."
996,"[{'role': 'user', 'content': 'This study inves..."
997,"[{'role': 'user', 'content': 'We have a friend..."
998,"[{'role': 'user', 'content': 'I have lost 55 p..."


In [11]:
def apply_chat_template(example, tokenizer):
    example = example['content']
    tokenized = tokenizer.apply_chat_template(example, tokenize=False)
    
    return {"content": tokenized}

In [12]:

from datasets import DatasetDict
from datasets import Dataset

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(df.iloc[:800]),
    "test": Dataset.from_pandas(df.iloc[800:])
})

In [13]:
datasets_train_test['train']['content'][0]

[{'content': 'Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ?',
  'role': 'user'},
 {'content': "hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but left untreated he slowly modifies the heart. according to heart rhythm, the normal rate is between 50-100 beat for minute. when it exceeds 100 we talk abou

In [14]:
raw_datasets = datasets_train_test.map(apply_chat_template,
                      fn_kwargs={"tokenizer": tokenizer},
                      batched=True,
                      desc="Running tokenizer on dataset")


Running tokenizer on dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
raw_datasets['train']['content'][0]

"<s>[INST] Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ? [/INST] hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but left untreated he slowly modifies the heart. according to heart rhythm, the normal rate is between 50-100 beat for minute. when it exceeds 100 we talk about sinus tachycardia. this migh

In [16]:
train_dataset = raw_datasets['train']
test_dataset = raw_datasets['test']

In [17]:
from transformers import BitsAndBytesConfig
import torch

quanitization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = "bfloat16",
)



In [30]:
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    fp16=True,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=5e-5,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs = 5,
    output_dir = "train/",
    report_to="tensorboard",
    save_strategy="epoch",
    logging_dir="train/logs/",
    save_total_limit=4,
    seed=42
)

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices


In [31]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
    lora_dropout=0.1,
    #bias=None,
    task_type="causal_lm",
)

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b7", load_in_8bit=True, device_map="cpu")

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/66/78/66784d641e353e03dbe21438011e73e2b0eefdfa725a4a42652362823616524b/145ae4b66381746c9438c63b6deb22a34d97000bba633bb6672fff2d1dcaf924?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1719101773&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTEwMTc3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Ni83OC82Njc4NGQ2NDFlMzUzZTAzZGJlMjE0MzgwMTFlNzNlMmIwZWVmZGZhNzI1YTRhNDI2NTIzNjI4MjM2MTY1MjRiLzE0NWFlNGI2NjM4MTc0NmM5NDM4YzYzYjZkZWIyMmEzNGQ5NzAwMGJiYTYzM2JiNjY3MmZmZjJkMWRjYWY5MjQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=RpO2nR4acoiqqi%7E71fx7UPKtcqF1wXGUuoU09ctpGtBb-%7EKrmqy%7EPTA7TF9f0pGUN-a4sfbulwm9u8nrNZvCgsD9dZoDzyRfc5PDrsU7IHOpO7qVTIUDZ12T61z99leKSBPkKnJOEffKCkm-E5Kuy%7ECLXgx1hStRj8s%7EpKXjbiWWZODehNfQKdAryjELbABnDvV5eDGEfX4UOjo2C%7ENe-Wi21OPNnzpFXpexFsCiNE

model.safetensors:  90%|######### | 3.11G/3.44G [00:00<?, ?B/s]

In [10]:
model.device

device(type='cpu')

In [11]:
#model.half()
model.dtype

torch.float16

In [33]:
tokenizer.pad_token = tokenizer.eos_token

In [34]:
trainer = SFTTrainer(
    model = model.module if isinstance(model, torch.nn.DataParallel) else model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config = peft_config,
    tokenizer = tokenizer,
    dataset_text_field = "content",
    packing = True,
    max_seq_length = 4096
    
)

Generating train split: 0 examples [00:00, ? examples/s]

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [23]:
trainer.train()

***** Running training *****
  Num examples = 52
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Training with DataParallel so batch size has been adjusted to: 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 65
  Number of trainable parameters = 12,582,912


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
